# **Full Stack: Upload a PDF for Edit Distance Matrix**

In [12]:
# pdf2image
!apt-get install poppler-utils
!pip install pdf2image
# OCR
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pdf2image
# Edit Distance
!pip install nltk
!pip install python-Levenshtein

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [13]:
from PIL import Image
import pdf2image
from pdf2image import convert_from_path
import time
import pytesseract
from google.colab import files
import IPython.display as display
from Levenshtein import distance as lev
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
import string
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
# from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Upload PDF to convert to JPEG

In [3]:
# Upload PDF if not already at current path
uploaded = files.upload()

Saving 8d133367-81c3-4f97-b24f-38f6ef3f11371.pdf to 8d133367-81c3-4f97-b24f-38f6ef3f11371.pdf


In [36]:
# For counting time
start_time = time.time()

# File name
images = convert_from_path('8d133367-81c3-4f97-b24f-38f6ef3f11371.pdf')

# Save the images page by page 
for i in range(len(images)):
    images[i].save('page'+ str(i) +'.jpg', 'JPEG')
    # To download the images, uncomment the line below
    # files.download('page{}.jpg'.format(i))

# Concatenate text from all pages in PDF
corpus = ""

for page in range(len(images)):
    im = Image.open('page{}.jpg'.format(str(page)))
    text = pytesseract.image_to_string(im)
    corpus += text

# print(corpus)

## Define the Keywords for Levenshein Matrix

In [37]:
# Define the Keywords for Levenshtein Distance
keywords = {
"AST_Task2":["covid",
"vaccine", 
"production", 
"mrna",
"manufacturing",
"chain",
"management", 
"operations", 
"rd", 
"research", 
"development",
"pfizer",
"moderna",
"biontech",
"lonza",
"sanofi", 
"curevac", 
"csl",
"luniabio", 
"knowledge", 
"gap"], 
"AST_Task3":
["covid",
"vaccine", 
"production", 
"mrna",
"manufacturing",
"chain",
"management", 
"operations", 
"rd", 
"research", 
"development",
"pfizer",
"moderna",
"biontech",
"lonza",
"sanofi", 
"curevac", 
"csl",
"luniabio", 
"knowledge", 
"gap"], 
"AST_Task4":
["alpha",
"sights",
"mr",
"ms",
"dr",
"prof",
"sir",
"madam",
"alphasights",
"dear",
"concern",
"yours",
"truly",
"faithfully",
"sincerely",
"regards",
"project",
"covid",
"manufacture",
"mrna",
"vaccine",
"consultation", 
"call",
"chain",
"management", 
"operations",
"rd", 
"research"
]}

## Edit Distance Matrix

In [38]:
# Lowercase
corpus = corpus.lower()

# Punctuations
corpus_alpha = "".join([char for char in corpus if char not in string.punctuation])

# Tokenization
words = word_tokenize(corpus_alpha)

# Stopword Filtering
stop_words = set(stopwords.words('english'))
filteredWords = [word for word in words if not word in stop_words]
# print(filteredWords)

# Lemmatization 
# lemmatizer = WordNetLemmatizer()
# List Unhashable
# filteredWords = lemmatizer.lemmatize(filteredWords)

count0words = []
count1words = []
count2words = []
distinctcount0words = []
count0=0
count1=0
count2=0

levMatrix = np.zeros((len(filteredWords),len(keywords['AST_Task2'])))
for text_word in range(len(filteredWords)):
  for key_word in range(len(keywords['AST_Task2'])):
    lev_dist = lev(filteredWords[text_word], keywords['AST_Task2'][key_word])
    levMatrix[text_word][key_word] = lev_dist
    if lev_dist == 0:
      count0words.append(filteredWords[text_word])
      count0 += 1
      if filteredWords[text_word] not in distinctcount0words:
        distinctcount0words.append(filteredWords[text_word])
    if lev_dist == 1:
      count1words.append(filteredWords[text_word])
      count1 += 1
    if lev_dist == 2:
      count2 += 1
      count2words.append(filteredWords[text_word])

print("--- %s seconds --- for %s pages" % (time.time() - start_time, len(images)))
print("-----Filtered Words-----")
print(filteredWords)
print("-----Edit Distance Matrix-----")
print(np.matrix(levMatrix))
print("Edit Distance Count: 0: {}, Unique 0: {}, 1: {}, 2: {}".format(len(count0words), len(distinctcount0words), len(count1words), len(count2words)))
print("-----MATCHED WORDS-----")
print(count0words)
print(distinctcount0words)
print(count1words)
print(count2words)

--- 5.187291145324707 seconds --- for 2 pages
-----Filtered Words-----
['knowledge', 'gap', 'technical', 'stages', 'mass', 'production', 'vaccines', 'alternative', 'methods', 'culture', 'stage', 'vaccine', 'production', 'process', 'design', 'meet', 'current', 'good', 'manufacture', 'practices', 'cgmp', 'implementation', 'cold', 'chain', 'freshkeeping', 'technology', 'safety', 'requirements', 'vaccine', 'producers', 'infected', 'virus', 'development', 'prospects', 'scope', 'application', 'vaccines', 'comparative', 'advantages', 'manufacturers', 'capital', 'input', 'expected', 'cost', 'initial', 'production', 'possibility', 'achieving', 'global', 'largescale', 'production', 'year', 'half', 'key', 'players', 'covid', 'vaccine', 'manufacturing', 'procurement', 'personnel', 'responsible', 'purchasing', 'important', 'raw', 'materials', 'culture', 'medium', 'syringes', 'vaccine', 'production', 'production', 'staff', 'works', 'shifts', 'produce', 'mrna', 'vaccine', 'low', 'pressure', 'package'

## Classification

In [39]:
# Use the "Optimized" Minimum number of keywords for classification
minKeys = 5

if len(distinctcount0words) >= minKeys:
  print("Not Spam")
else: 
  print("Spam")

Not Spam
